# Análisis del poder predictivo de las variables

In [ ]:
# Importando las librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

from sklearn.decomposition import PCA
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.utils import to_time_series_dataset

In [ ]:
TARGET = "Symptoms_bef_Dg_DIC_wo_vagal" #variable objetivo
n_clusters = 2 #definimos el número de clusters, en este caso son 2, para pacientes sintomáticos y asintomáticos

time_series = pd.read_excel("./Biomarkers_12leads.xlsx") #lectura del dataset que contiene las series temporales
time_series.fillna(0, inplace=True) # reemplazando los valores faltantes por 0

# Lectura del dataset que contiene la etiqueta
labels_df = pd.read_excel("./dsBiomarkersPlusClinical.xlsx")[
    ["BH", TARGET]
]
y_train = labels_df.groupby("BH").agg("mean") #agrupando el dataset por el identificador de paciente y obteniendo la etiqueta

In [ ]:
F1_metrics = {} #diccionario para almacenar el score F1 para cada variable

features = time_series.columns #obteniendo una lista de todas las variables disponibles
features.drop("BH") #eliminando el identificador de paciente de la lista de variables

for feature in features: #iterando por cada variable disponible para obtener la métrica F1 entrenando un modelo con cada una
    time_series_list = [] #definiendo una lista para almacenar las series temporales

    for bh in y_train.index: #iterando por cada uno de los pacientes
        #para cada paciente se almacena la serie temporal de la variable en la lista time_series_list
        time_series_list.append(
            time_series[time_series["BH"] == bh][feature].to_numpy()
        )

    time_series_array = to_time_series_dataset(np.array(time_series_list)) #transformando la lista de series temporales de los pacientes en un array y luego en un dataset de series temporales

    X_train = TimeSeriesScalerMeanVariance().fit_transform(time_series_array) #escalando las series temporales de los pacientes con la técnica de TimeSeriesScalerMeanVariance

    ## DTW dinamic time warping
    ts_km = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", verbose=False) #creamos un modelo para clusterización de series temporales TimeSeriesKMeans
    ts_km.fit(X_train, y_train[TARGET]) #entrenamos un modelo con las series temporales escaladas de la variable

    cluster_assignments = ts_km.labels_ #obtenemos los resultados de la clasificación realizada por el modelo

    F1_metrics[feature] = f1_score(y_train[TARGET], cluster_assignments) #obtenemos el score F1 de la clasificación realizada con la variable y guardamos el resultado en el diccionario junto al nombre de la variable


In [ ]:
F1_metrics #el diccionario contiene las variables disónibles junto al score F1 obtenido del modelo entrenado sólo con dicha variable, la métrica muestra el poder predictivo que tiene la misma

# Gráficos de series temporales

In [ ]:
#lista con las mejores variables
best_variables = [
    "ST_0_V5", #seleccionada
    # "PR_III",
    # "LP_QRSd_III",
    "ST_0_I", #seleccionada
    "LP_LAS_III", #seleccionada
    # "LP_QRSd_II",
    # "ST_slope_aVL",
    # "LP_LAS_V6",
    "LP_RMS40_V4", #seleccionada
]

time_series_list = [] # lista para almacenar las series temporales

for bh in y_train.index: # iterando por cada paciente
    time_series_list.append(
        time_series[time_series["BH"] == bh][best_variables].to_numpy() #agregamos las series temporales de las variables seleccionadas del paciente a la lista
    )

time_series_array = to_time_series_dataset(np.array(time_series_list)) #transformando la lista de series temporales de los pacientes en un array y luego en un dataset de series temporales

X_train = TimeSeriesScalerMeanVariance().fit_transform(time_series_array) #escalando las series temporales de los pacientes con la técnica de TimeSeriesScalerMeanVariance


ts_km = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", verbose=False) #creamos un modelo para clusterización de series temporales TimeSeriesKMeans
ts_km.fit(X_train, y_train[TARGET]) #entrenamos un modelo con las series temporales escaladas de la variable


plt.figure(figsize=(15, 10)) #Creando la figura para el gráfico, con medidas 15*10

for yi in range(n_clusters): #Iterando por cada uno de los clusters, para pacientes sintomáticos y asintomáticos
    plt.subplot(2, n_clusters, yi + 1) #creamos los subplots, para cada categoría

    for xx in X_train[y_train[TARGET] == yi]: #para cada paciente de cada categoría graficamos las series temporales aplanadas con una opacidad de 0.15
        plt.plot(xx.ravel(), "k-", alpha=0.15) 

    plt.plot(ts_km.cluster_centers_[yi].ravel(), "r-") #graficamos en rojo el centroide de los clusters

    plt.ylim(-15, 15) # definiendo los límites verticales del gráfico, para que se puedan ver alineados

    plt.xlabel("Observaciones") #agregando etiquetas al eje x e y
    plt.ylabel("Variables")

    if yi == 0: #si el cluster es 0, se agrega el texto ASINTOMÁTICO
        plt.text(0.15, 0.95, "ASINTOMÁTICO" , transform=plt.gca().transAxes)

    elif yi == 1: #si el cluster es 1, se agrega el texto SINTOMÁTICO
        plt.text(0.15, 0.95, "SINTOMÁTICO" , transform=plt.gca().transAxes)

    else: #en cualquier otro caso se agrega el texto ERROR
        plt.text(0.15, 0.95, "ERROR" , transform=plt.gca().transAxes)